<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BCS" data-toc-modified-id="BCS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>BCS</a></span></li><li><span><a href="#VQD" data-toc-modified-id="VQD-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>VQD</a></span></li><li><span><a href="#Hamiltonian-Simulation" data-toc-modified-id="Hamiltonian-Simulation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hamiltonian Simulation</a></span></li></ul></div>

# BCS

Using the Paper by [Wu et al.](https://arxiv.org/abs/quant-ph/0108110), we have that:

The BCS hamiltonian is:

$$H_{BCS} = \sum_{m=1}^N \frac{\epsilon_m}{2} ( n_m^F + n_{-m}^F ) + \sum_{m,k=1}^N V^{+}_{ml} c^\dagger_m c^\dagger_{-m} c_{-l}c_l$$

Where $n_{\pm m}^F = c^\dagger_{\pm m} c_{\pm m}$, and the matrix elements $V^+_{ml} = \langle m, -m| V | l, -l \rangle$.

Which is mapped into:

$$H_P = \sum_{m=1}^N \frac{\epsilon_m}{2} \sigma^Z_m + \sum_{r= \pm l} \sum_{l>m=1}^N \frac{V_{ml}^r}{2} ( \sigma^x_m \sigma^x_l + r \sigma^y_m \sigma^y_l )$$

where $\epsilon_m = \epsilon_m + V^+_{mm}$ and $V^-_{ml} = 0$ for the BCS Hamiltonian, thus:

$$H_P = \sum_{m=1}^N \frac{\epsilon_m}{2} \sigma^Z_m +  \sum_{l>m=1}^N \frac{V_{ml}}{2} ( \sigma^x_m \sigma^x_l +  \sigma^y_m \sigma^y_l )$$

$l \ \mathrm{and} \ m$ now denote both state and qubit indices. In this case the qubit state space $H_P = Span \{ |0 \rangle, \sigma^+_m |0 \rangle, \sigma_l^+ \sigma_m^+ |0 \rangle, \dots   \}$ is mapped into a subspace where $n_m^F = n^F_{-m}$. Considering $V_{ml} = V$, we have that:

$$
H_p = \hbar \bigg[ \sum_{m=1}^N \frac{\epsilon_m}{2} \sigma_z^m + \frac{V}{2} \sum_{l>m=1}^N (\sigma_x^m \sigma_x^l + \sigma_y^m \sigma_y^l) \bigg]
$$
[Aqua Operator Flow Reference](https://github.com/dongreenberg/aqua_talks/blob/master/Understanding%20Aqua's%20Operator%20Flow.ipynb)

In [9]:
import numpy as np
from qiskit.aqua.operators import I, X, Y, Z
from VQD import VQD
from utils import classical_solver

%load_ext autoreload
%autoreload 2

def BCS_hamiltonian(epsilons, V):
    return ((1/2*epsilons[0]*I^Z) + (1/2*epsilons[1]*Z^I) + V*(1/2*(X^X) + 1/2*(Y^Y)))

In [16]:
EPSILONS = [2,1]
V = 2
hamiltonian = BCS_hamiltonian(EPSILONS, V)
print(hamiltonian)

SummedOp([
  IZ,
  0.5 * ZI,
  XX,
  YY
])


In [17]:
eigenvalues = classical_solver(hamiltonian)
print(f"Eigenvalues: {eigenvalues}")

Eigenvalues: [-2.06155281 -1.5         1.5         2.06155281]


# VQD

In [18]:
from qiskit import BasicAer
from qiskit.aqua.components.optimizers import COBYLA

optimizer = COBYLA(maxiter=1000)
backend = BasicAer.get_backend('qasm_simulator')

Algo = VQD(hamiltonian=hamiltonian,
            n_excited_states=1,
            beta=1.,
            optimizer=optimizer,
            backend=backend)

Algo.run()      
vqd_energies = Algo.energies

Calculating excited state 0
Calculating excited state 1


In [19]:
vqd_energies

[-1.8505859375, -1.6720625]